1. Utilizando Python e Numpy, implemente uma Multi-layer Perceptron (MLP) com uma camada oculta de largura 2 (i.e., uma Perceptron com duas unidades ocultas), que aproxima função “ou exclusivo” (XOR) definida como XOR : {0, 1}² → {0, 1}.
<img src="mlp_xor.png"><img/>

In [165]:
import numpy as np

2. Considere que apenas a camada oculta tem uma função de ativação (adote ReLU) (i.e., a camada de saída tem
ativação identidade). Otimize a MLP com uma função de perda que penaliza o quadrado do erro de estimação.

In [166]:
def ReLU(x):
    return max(0, x)

def I(x):
    return x

def funcao_perda(saida_esperada, saida_real):
    return (saida_real - saida_esperada) ** 2

3. Inicialize os coeficientes das matrizes de pesos Ω de tamanho N × M com os valores Ωij = N(0, √(2/N)) (He initialization). Inicialize os coeficientes dos vetores de viéses β com um valor constante pequeno (e.g., βk = 0,1).

In [167]:
def He_initialization(n, m):
    pesos = np.random.randn(n, m) * np.sqrt(2 / n)
    return pesos

4. Considere o método do gradiente descendente para otimização. Determine as derivadas da função de perda em função de cada parâmetro e as implemente manualmente. Identifique os gradientes textualmente.

In [168]:
def backward_pass(entradas, saida_correta, saida_aproximada, saida_pesos, oculta_saida, oculta_saida_linear):
    """
    Otimiza os parâmetros da MLP usando o método do gradiente descendente.
    >argumentos:
    - entradas = vetor de entradas
    - saida_correta = saida correta esperada
    - saida_aproximada = saida aproximada pela MLP
    - saida_pesos = matriz de pesos da camada de saída
    - oculta_saida: saida da oculta depois de ser aplicada a uma função de ativação
    - oculta_saida_linear: saida da oculta antes de ser aplicada a uma função de ativação
    >retorno: [gradiente_vies_saida, gradiente_peso_saida, gradiente_vies_oculta, gradiente_peso_oculta]
    """

    # gradiente do viés da camada de saída
    # é a derivada da função de perda em relação aos pesos da camada de saída
    gradiente_vies_saida = 2 * (saida_aproximada - saida_correta)
    
    # gradiente dos pesos da camada de saída
    # é o produto  entre as ativações da camada oculta e o erro da camada de saída
    gradiente_peso_saida = oculta_saida.T @ gradiente_vies_saida
    
    # gradiente dos viéses da camada oculta
    # Erro propagado para a camada oculta.
    derivada_ReLU = (oculta_saida_linear > 0).astype(int)
    gradiente_vies_oculta = (gradiente_vies_saida @ saida_pesos.T) * derivada_ReLU
    
    # gradiente dos pesos da camada oculta
    # erro propagado para a camada oculta
    gradiente_pesos_oculta = entradas.T @ gradiente_vies_oculta
    
    return gradiente_vies_saida, gradiente_peso_saida, gradiente_vies_oculta, gradiente_pesos_oculta
    
    
def atualiza_parametro(parametro_antigo, taxa_aprendizado, gradiente):
    return parametro_antigo - taxa_aprendizado * gradiente

5. Avalie experimentalmente o impacto de retreinar a MLP. Por fim, determine uma seed para os experimentos seguintes. Descreva os resultados.

In [169]:
# hiperparâmetros
epocas = 3000
taxa_aprendizado = 0.01
np.random.seed(80)

# pesos e viéses da camada oculta
h_pesos = He_initialization(2, 2)
h_vieses = np.array([[0.1, 0.1]])

# pesos e viéses da camada de saída
yhat_pesos = He_initialization(2, 1)
yhat_vieses = np.array([[0.1]])

# entradas e saídas esperadas da MLP
entradas = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
saidas_esperadas = np.array([0, 1, 1, 0])


# machine.learn() 
for i in range(epocas):
    perda_total = 0
    
    for entrada_x, saida_y in zip(entradas, saidas_esperadas):    
        entrada_x = entrada_x.reshape(1, -1)    # (1, 2)
        saida_y = np.array([[saida_y]])         # (1, 1)

            
        # calcula a saida da camada oculta
        saida_oculta_linear = entrada_x @ h_pesos
        saida_oculta_linear += h_vieses
        saida_oculta = np.vectorize(ReLU)(saida_oculta_linear)
        
        # calcula a saida da camada de saida
        saida = saida_oculta @ yhat_pesos + yhat_vieses

        
        #calcula a perda
        perda = funcao_perda(saida_y, saida)
        perda_total += perda
        
        # calcula os gradientes
        gradientes = backward_pass(entrada_x, saida_y, saida, yhat_pesos, saida_oculta, saida_oculta_linear)
        
        # atualiza os parâmetros
        yhat_vieses = atualiza_parametro(yhat_vieses, taxa_aprendizado, gradientes[0])
        yhat_pesos = atualiza_parametro(yhat_pesos, taxa_aprendizado, gradientes[1])
        h_vieses = atualiza_parametro(h_vieses, taxa_aprendizado, gradientes[2])
        h_pesos = atualiza_parametro(h_pesos, taxa_aprendizado, gradientes[3])
    
    if(i + 1) % 500 == 0:
        perda_medio = perda_total / len(entradas)
        print(f"Época: {i + 1}, loss médio: {perda_medio.item():.6f}")

print("\n--- Testando a Rede Treinada ---")
for x_exemplo, y_exemplo in zip(entradas, saidas_esperadas):
    entrada = x_exemplo.reshape(1, 2)
    saida_oculta_linear = entrada @ h_pesos + h_vieses
    saida_oculta = np.vectorize(ReLU)(saida_oculta_linear)
    saida = saida_oculta @ yhat_pesos + yhat_vieses
    # Arredonda a previsão para 0 ou 1 para uma comparação clara
    print(f"Entrada: {x_exemplo}, Saída Esperada: {y_exemplo}, Previsão da Rede: ({saida[0][0]:.4f})")
        
        

Época: 500, loss médio: 0.151750
Época: 1000, loss médio: 0.103305
Época: 1500, loss médio: 0.007572
Época: 2000, loss médio: 0.000007
Época: 2500, loss médio: 0.000000
Época: 3000, loss médio: 0.000000

--- Testando a Rede Treinada ---
Entrada: [0 0], Saída Esperada: 0, Previsão da Rede: (0.0000)
Entrada: [0 1], Saída Esperada: 1, Previsão da Rede: (1.0000)
Entrada: [1 0], Saída Esperada: 1, Previsão da Rede: (1.0000)
Entrada: [1 1], Saída Esperada: 0, Previsão da Rede: (0.0000)
